# Meeting summary pipeline

Here we will try to implement meeting summary pipeline espesialy audio to text summary.

Recording is taken from this [dataset](https://huggingface.co/datasets/huuuyeah/meetingbank)
Audio files can be found [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main)

For testing purpose audio file is taken localy into '/extra' folder

In [46]:
# Imports
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from huggingface_hub import login
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import numpy as np
import torch


In [37]:
# Load environment variables
load_dotenv() 

# Add project root to Python path and change working directory
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))
os.chdir(project_root)  # Change working directory to project root

In [19]:
# Sign in to HuggingFace Hub

hf_token = os.environ.get('HF_TOKEN')
if hf_token is None:
    raise ValueError("HF_TOKEN is not set")
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Brifly about models we are using here:

**distil-medium.en** - is a distilated whisper model that is trained using Robust Knowledge Distillation. Distilated model will be not only smaller by 49% but also 6 times faster.
- More about Robust Knowledge Distillation you can read [here](https://arxiv.org/abs/2311.00430)
- Model can be found [here](https://huggingface.co/distil-whisper/distil-medium.en)



In [ ]:
# Constants

ASR_MODEL = "distil-whisper/distil-medium.en"

audio_file_name = "extra/denver_meeting_rec.mp3"

## Audio To Text

Let's start from our ASR. We will use distil-whisper using HF transformers pipelines. As long as our audio is longer than 30-seconds (when *Short-Form Transcription* can be used) we will use *Long-Form Transcription*. 

Small note from model page:
> Distil-Whisper uses a chunked algorithm to transcribe long-form audio files (> 30-seconds).
> In practice, this chunked long-form algorithm is 9x faster than the sequential > algorithm proposed by OpenAI in the Whisper paper

In [39]:
# Initialize ASR model
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    ASR_MODEL, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(ASR_MODEL)

asr_pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15, # This will enable Long-Form Transcription
    # batch_size=16, # for batch processing
    torch_dtype=torch_dtype,
    device=device,
)

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [40]:
result = asr_pipe(audio_file_name)

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.


In [45]:
print(result['text'])


 kind of the confluence of this whole idea of the confluence week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue so that is the reason that the back of the logo is considered water. So let you see the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous Peoples Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United States of America, to the Republic of which it stands, one nation unde